<a href="https://colab.research.google.com/github/divyanshkulshrestha23/Outlier-Detection/blob/main/Outlier_Detection_Isolation_Forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os, cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest
from sklearn.decomposition import PCA
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import Model

In [ ]:
location = "Enter File Path Here"

In [ ]:
image_paths = [os.path.join(location, f) for f in os.listdir(location) if f.endswith('jpeg')]
features = []
raw_images = []

In [ ]:
base_model = MobileNetV2(weights='imagenet', include_top=False, pooling='avg')
model = Model(inputs=base_model.input, outputs=base_model.output)

In [ ]:
for path in image_paths:
    img = cv2.imread(path)
    if img is None:
        continue

    img = cv2.resize(img, (224, 224))
    img_array = img_to_array(img)
    img_array = preprocess_input(img_array)
    img_array = np.expand_dims(img_array, axis=0)

    feat = model.predict(img_array, verbose=0).flatten()
    features.append(feat)
    raw_images.append(path)

features = np.array(features)

In [ ]:
pca = PCA(n_components=50)
X_pca = pca.fit_transform(features)

In [ ]:
percentage = "Enter Estimated Percentage of Outliers Here"

In [ ]:
clf = IsolationForest(contamination=0.04, random_state=42)  # ~3% outliers
preds = clf.fit_predict(X_pca)

In [ ]:
outlier_indices = np.where(preds == -1)[0]
outliers = [raw_images[i] for i in outlier_indices]

In [ ]:
import math

num_outliers = len(outlier_indices)
cols = 5  # Number of images per row
rows = math.ceil(num_outliers / cols)

plt.figure(figsize=(cols * 3, rows * 3))  # Scale size by number of rows/cols

for i, idx in enumerate(outlier_indices):
    img = cv2.imread(raw_images[idx])
    img = cv2.cvtColor(cv2.resize(img, (128, 128)), cv2.COLOR_BGR2RGB)
    plt.subplot(rows, cols, i + 1)
    plt.imshow(img)
    plt.title(f"Outlier #{i + 1}")
    plt.axis('off')

plt.suptitle("All Detected Outlier Images", fontsize=16)
plt.tight_layout()
plt.show()
